<h1 style="text-align:center">Creación de modelo de analisis de sentimientos</h1>

In [1]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
import matplotlib.pyplot as plt
import os
spark = sparknlp.start()

print(f"Versión de spark: {spark.version}")
print(f"Versión de sparknlp: {sparknlp.version()}")

Versión de spark: 3.1.1
Versión de sparknlp: 3.0.1


In [2]:
MODEL_NAME = 'webinar-sentiment-dl'
LOGS_PATH = os.path.join('.', 'logs', MODEL_NAME)

In [3]:
dataframe = spark \
    .read \
    .option("header", True) \
    .csv("./archive/Tweets.csv")

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentiment = SentimentDLApproach() \
    .setInputCol("sentence_embeddings") \
    .setOutputCol("sentiment") \
    .setBatchSize(64) \
    .setDropout(.3) \
    .setLr(.025) \
    .shufflePerEpoch(True) \
    .setEnableOutputLogs(True) \
    .setOutputLogsPath(LOGS_PATH) \
    .setValidationSplit(.25)


tfhub_use download started this may take some time.
Approximate size to download 923,7 MB
[ | ]

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


[OK!]

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-b370def6d564>", line 5, in <module>
    use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/sparknlp/annotator.py", line 2202, in pretrained
    return ResourceDownloader.downloadModel(UniversalSentenceEncoder, name, lang, remote_loc)
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/sparknlp/pretrained.py", line 41, in downloadModel
    j_obj = _internal._DownloadModel(reader.name, name, language, remote_loc, j_dwn).apply()
  File "/home/luciano/anaconda3/envs/spark-webinar/lib/python3.9/site-packages/sparknlp/internal.py", line 176, in __init__
    super(_DownloadModel

Py4JError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel